<a href="https://colab.research.google.com/github/jennysands/Software-Engineering/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import csv
import os #for file navigation

ROOT_PATH = os.path.dirname(os.path.abspath(__file__))
os.chdir(ROOT_PATH)             #ensure all following commands are running relative to the location of this file

#fairly simple way to get all filepaths of files in the folder of the code file and all subfolders
#then easily filter-able given the consistent syntax to get specific files
def getfilepaths(inputdir):
    filelist = list()
    def listfiles(rootdir): #recursive function to get filenames of everything in the root path and every subfolder
        for fileorfoldername in os.scandir(rootdir):
            if fileorfoldername.is_dir():
                filelist.append(fileorfoldername.path)
                listfiles(fileorfoldername)
            else:
                filelist.append(fileorfoldername.path)
    listfiles(inputdir)
    return filelist

#stuff specifically for the small audio dataset without meta data, "\" might be different if the GPU machine is running linux?
X_train = getfilepaths(ROOT_PATH + "\insectsound_MariusFaißMSc\ManualTrain")
X_test = getfilepaths(ROOT_PATH + "\insectsound_MariusFaißMSc\ManualTest")
X_val = getfilepaths(ROOT_PATH + "\insectsound_MariusFaißMSc\ManualValidation")

# New Section

In [ ]:
import pandas as pd
import os
import torch
import torchaudio
from torchaudio import transforms
from torchaudio.utils import download_asset
import random
import matplotlib.pyplot as plt
#Loading the data

listed_files_train = [] #Creating a list to remove inappropriate data
current_path_train = "C:/Users/carlv/Downloads/insectsound_MariusFaißMSc/ManualTrain"
x_train =  os.scandir("C:/Users/carlv/Downloads/insectsound_MariusFaißMSc/ManualTrain")  
for train_path in x_train:
    listed_files_train.append(train_path)
listed_files_train.remove(listed_files_train[0])

#Creating the main class, which will have necessary methods

class AudioUtil():
    
    #loading and reading the files
   
    @staticmethod
    def file_open(audio_file):
        sig, sr = torchaudio.load(audio_file)
        return sig, sr
    
    #Equalizing the sample sizes
    
    @staticmethod
    def pad_trunc(aud, max_ms):
        sig, sr = aud
        num_rows, sig_len = sig.shape
        max_len = sr//1000 * max_ms

        if (sig_len > max_len):
            sig = sig[:,:max_len]

        elif (sig_len < max_len):
            pad_begin_len = random.randint(0, max_len - sig_len)
            pad_end_len = max_len - sig_len - pad_begin_len

            pad_begin = torch.zeros((num_rows, pad_begin_len))
            pad_end = torch.zeros((num_rows, pad_end_len))

            sig = torch.cat((pad_begin, sig, pad_end), 1)
      
        return (sig, sr)
    
    #Shifting time. (For now, I don't know what I should add in the second parameter when testing)
    
    @staticmethod
    def time_shift(aud, shift_limit):
        sig,sr = aud
        _, sig_len = sig.shape
        shift_amt = int(random.random() * shift_limit * sig_len)
        return (sig.roll(shift_amt), sr)
    
    # Mel spectrogram
    
    @staticmethod
    def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
        sig,sr = aud
        top_db = 80

        # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
        spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)

        # Convert to decibels
        spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
        return (spec)
    
    # Augemented Mel spectrogram
    
    @staticmethod
    def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
        _, n_mels, n_steps = spec.shape
        mask_value = spec.mean()
        aug_spec = spec

        freq_mask_param = max_mask_pct * n_mels
        for _ in range(n_freq_masks):
            aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

        time_mask_param = max_mask_pct * n_steps
        for _ in range(n_time_masks):
            aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)

        return aug_spec
#Testing the output
for link_train in listed_files_train:
    open_link_train = AudioUtil.file_open(link_train)
    link_train_resize = AudioUtil.pad_trunc(open_link_train,10000)
    link_train_shifted = AudioUtil.time_shift(link_train_resize,1000)
    mel_spect = AudioUtil.spectro_gram(aud = link_train_resize)
    augmented_spect = AudioUtil.spectro_augment(spec = mel_spect)
    augmented_spect = augmented_spect.squeeze(0)
    #print(mel_spect)
    #print(augmented_spect) 
print(augmented_spect.shape)
plt.figure()
plt.plot(augmented_spect)
plt.show()

hello
